In [40]:
# Load modules
%run -i scripts/login.py
%run -i scripts/location.py
%run -i scripts/weight.py
%run -i scripts/diarrhea_observations.py

### webvitals.py

In [24]:
# Example query for testing
query = ['47455','47510' ,'40459']

In [41]:
import selenium
import datetime
import getpass
import time
import os

import numpy as np
import pandas as pd
import dateutil.parser as dparser

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException

from tqdm import tqdm

from jproperties import Properties

from webdriver_manager.firefox import GeckoDriverManager

from bs4 import BeautifulSoup

# Install Geckodriver
driver = webdriver.Firefox(executable_path=GeckoDriverManager(version="v0.20.0").install())

# # Enter options
# options = FirefoxOptions()
# options.add_argument("--headless")
# driver = webdriver.Firefox(options=options)

# Go to login page
driver.get("https://davos.primate.ucdavis.edu/login/")

login()

print('Running location query')
time.sleep(2)
location(query)

print('Running diarrhea observation query')
time.sleep(2)
diarrhea_observations(query)

print('Running weight query')
time.sleep(2)
weight(query)

# def snomed(ids):
#     data = pd.DataFrame()
    
#     for i in ids:
#         # Go to snomed in Webvitals
#         driver.find_element_by_name("query_input").send_keys('47455')
#         driver.find_element_by_name("submit").click()
        
#         xpath="/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[8]/a"
#         driver.find_element_by_xpath(xpath).click()

#         # Extract html table
#         xpath="/html/body/table[2]/tbody/tr/td[1]/html:errors/center[1]/table"
#         tableelement= (
#             WebDriverWait(driver,10)

#             .until(EC.visibility_of_element_located((By.XPATH, xpath)))
#             .get_attribute('outerHTML')
#         )        
#         table = pd.read_html(str(tableelement))[0]

#         # Add column to specify MMU number
#         table['MMU'] = '47455'
#         first_column = table.pop('MMU')
#         table.insert(0, 'MMU', first_column)

#         # Append dataframes into one dataframe
#         data = data.append(table, ignore_index=True)
#         print(data)



print('driver Title:',driver.title)
print('Driver name:',driver.name)
print('Driver URL:',driver.current_url)
driver.quit()

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Driver [C:\Users\Noah Siegel\.wdm\drivers\geckodriver\win64\v0.20.0\geckodriver.exe] found in cache
INFO:WDM:Driver [C:\Users\Noah Siegel\.wdm\drivers\geckodriver\win64\v0.20.0\geckodriver.exe] found in cache


'webvitals_config.properties' found!
[+] Login successful!

Running location query


100%|████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.82it/s]


Running diarrhea observation query


100%|████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.51it/s]


Running weight query


100%|████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.66it/s]


driver Title: CNPRC Web VITALS ver. 2.5.3
Driver name: firefox
Driver URL: https://davos.primate.ucdavis.edu/webVitals/queryAnimal.do?animalID=40459&dPage=WeightsTB
